In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display

First, we create the coordinate arrays for the 3D plot. The plot will be shown in the space of Eigenvalues $\sigma_1$, $\sigma_2$, $\sigma_3$.

In [2]:
a = 1 # show space -a <= sigma_i <= a
n = 31 # number of data points along each axis, i.e. resolution of the plot

# vectors along each axis with shape (n,)
s1 = s2 = s3 = np.linspace(-a,a,n)

# 3d grid of values for each coordinate with shape (n,n,n)
S1,S2,S3 = np.meshgrid(s1,s2,s3)

# reformat the coordinates as vectors
S1 = S1.flatten()
S2 = S2.flatten()
S3 = S3.flatten()


Next, we define the functions we want to visualize depending on `S1`, `S2`, `S3`

In [3]:
R = np.sqrt(S1**2+S2**2+S3**2)

J1 = S1 + S2 + S3
J2 = S1*S2 + S2*S3 + S1*S3
J3 = S1*S2*S3

def j2dev(s1, s2, s3):
    A = np.diag([s1, s2, s3])
    Adev = A - 1/3*np.trace(A)*np.eye(3)
    
    return np.sum(Adev*Adev)

Mises = np.array([j2dev(s1,s2,s3) for s1,s2,s3 in zip(S1,S2,S3)])

data = {'J1': J1, 'J2': J2, 'J3': J3, 'Mises': Mises}

colors = {'J1': 'rgb(21,104,156)',
          'J2': 'rgb(237,179,20)',
          'J3': 'rgb(153,148,194)',
          'Mises': 'rgb(35,100,52)'}

If you have specified data as a dictionary, the nxt cell should produce nice plots without changes

In [27]:
min_val, max_val = -1, 1
initial_val = 0.5


# dictionaries for checkboxes, sliders, plots and color patches
checkboxes = {}
sliders = {}
traces = {}
color_patches = {}

# the line for the volumetric stress
checkboxes['line_vol'] = widgets.Checkbox(description='Volumetrischer Zustand', value=True)

traces['line_vol'] = go.Scatter3d(x=s1,
                                  y=s2,
                                  z=s3,
                                  mode='lines',
                                  line=dict(color="#000000", width=3))

color_patches['line_vol'] = widgets.HTML("""<svg height="20" width="20">
 <line x1="0" y1="10" x2="20" y2="10" style="stroke:rgb(0,0,0);stroke-width:1.5" />
</svg>""")

for key, vals in data.items():
    color = colors.get(key, 'rgb(150,150,150)')
    
    color_patches[key] = widgets.HTML(value=f"""<svg width="20" height="20">
<rect width="20" height="20" style="fill:{color};stroke-width:0;stroke:#000000" />
</svg>""")
    
    checkboxes[key] = widgets.Checkbox(description=key, value=True)
    
    sliders[key] = widgets.FloatSlider(value=initial_val,
                                       min=min_val,
                                       max=max_val,
                                       step=0.1,
                                       description='Wert:',
                                       disabled=False,
                                       continuous_update=True,
                                       orientation='horizontal',
                                       readout=True,
                                       readout_format='.1f',
                                       )
    
    traces[key] = go.Isosurface(x=S1,
                                y=S2,
                                z=S3,
                                value=vals,
                                isomin=initial_val,
                                isomax=initial_val,
                                opacity=1,
                                surface=dict(show=True,count=1, fill=1),
                                caps=go.isosurface.Caps(z=dict(show=False),
                                                        x=dict(show=False),
                                                        y=dict(show=False),
                                ),
                                colorscale=[[0, color],[1, color]],
                                showscale=False
                                )
    
# inverse mapping for checkboxes
checkboxes_inv = dict(zip(checkboxes.values(), checkboxes.keys()))
sliders_inv = dict(zip(sliders.values(), sliders.keys()))

# index of each trace
trace_indices = {key: i for i, key in enumerate(traces.keys())}

fig = go.FigureWidget(data=list(traces.values()))

fig.update_layout(scene = dict(xaxis_title='sigma1',
                               yaxis_title='sigma2',
                               zaxis_title='sigma3',
                               xaxis_tickfont_size=14,
                               yaxis_tickfont_size=14,
                               zaxis_tickfont_size=14,
                               xaxis_title_font=dict(size=20),
                               yaxis_title_font=dict(size=20),
                               zaxis_title_font=dict(size=20),
                              ),
                  width=800, height=800)



def update_cb(change):
    key = checkboxes_inv[change.owner]
    trace = fig.data[trace_indices[key]]
    trace.visible = change['new']
    
def update_slider(change): 
    with fig.batch_update():
        key = sliders_inv[change.owner]
        trace = fig.data[trace_indices[key]]
        
        trace.isomin = change['new']
        trace.isomax = change['new']

for cb in checkboxes.values():
    cb.observe(update_cb, names="value")
    
for slider in sliders.values():
    slider.observe(update_slider, names="value")
    

rows = [widgets.HBox([color_patches['line_vol'], checkboxes['line_vol']])]
for key in data.keys():
    rows.append(widgets.HBox([color_patches[key], checkboxes[key], sliders[key]]))

rows.append(fig)
    
display(widgets.VBox(rows), renderer="browser")

